In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import matplotlib.colors as colors
import matplotlib.patches as mpatches
import cmocean
plt.style.use('../misc/acgc.mplstyle')

import json
from matplotlib.colors import LinearSegmentedColormap
with open('../misc/sargassum.json', 'r') as f:
    sargassum = LinearSegmentedColormap("sargassum", json.load(f))

from plotting_functions import *
from analysis_functions import *

domain_bounds = get_domain_bounds()

In [ ]:
ds = xr.open_mfdataset('../model_output/v14.3/Baseline*.nc', combine='by_coords')
ds = ds.sel(time=slice('2015-05-01', '2016-04-30'))
ds

In [4]:
# ----------------------------------------
# Load evaluation data - deposition and concentrations
# ----------------------------------------

# Wet Deposition
MDN = pd.read_csv('../misc/MDN-ALL-A-s.csv')
MDN = MDN[MDN['siteid'].str.contains('AK')]
MDN = MDN[MDN['HgDep'] != -9.]

MDN_metadata = pd.DataFrame({'siteid':['AK00', 'AK02', 'AK04', 'AK05', 'AK06','AK96', 'AK98', 'AK99'], 
                             'name':  ['Dutch Harbor', 'Juneau', 'Nome', 'Glacier Bay National Park - Bartlett Cove', 'Gates of the Arctic National Park - Bettles', 'Toolik Field Station', 'Kodiak', 'Ambler'], 
                             'lat':   [53.8454, 58.5139, 64.5055, 58.4566, 66.906, 68.6257, 57.7189, 67.0931], 
                             'lon':   [-166.5048, -134.7843, -165.396, -135.8674, -151.683, -149.6069, -152.5617, -157.8689],
                             'elev':  [58, 25, 15, 2, 630, 730, 7, 88], # elevation in meters
                             'start_date': ['2009-09-25', '2016-12-27', '2013-09-25', '2010-03-16', '2008-11-04', '2017-10-10', '2007-09-18', '2004-05-12'],
                             'end_date':   ['2015-09-29', 'present', '2015-09-29', '2013-05-21', '2015-10-27', 'present', '2019-05-06', '2005-08-09']})

# Atmospheric Concentrations
AMNet = pd.read_csv('../misc/AMNet-ak03-h.csv')
AMNet = pd.concat([AMNet, pd.read_csv('../misc/AMNet-ak95-h.csv')])

AMNet.replace(-9., np.nan, inplace=True)
AMNet['collStart'] = pd.to_datetime(AMNet['collStart'])
AMNet['collEnd'] = pd.to_datetime(AMNet['collEnd'])
AMNet['time'] = AMNet['collStart'] + (AMNet['collEnd'] - AMNet['collStart'])/2

AMNet_metadata = pd.DataFrame({'SiteID':['AK95', 'AK03'], 'name':['Utqiagvik', 'Denali National Park'], 
                               'lat':[71.323,63.7232], 'lon':[-156.6114,-148.9676],
                               'start_date': ['2021-10-08','2014-03-10'],'end_date':['present', '2018-12-21']})

# Dry Deposition
Obrist = pd.DataFrame({'site':['Toolik'], 'lat':[68.63], 'lon':[-149.6], 'HgDep':[6.5+2.5], 'Source':['Obrist et al., 2017']})

In [5]:
def get_monthly_concentration(df):
    SiteID = df['SiteID'].unique()[0]
    # note left 
    df_mean = df.resample('M', on='time').mean(numeric_only=True)
    df_std  = df.resample('M', on='time').std(numeric_only=True)
    df_mean.reset_index(inplace=True)
    df_std.reset_index(inplace=True)
    for col in ['PBM','GOM','GEM']:
        df_std.rename(columns={col:col+'_std'}, inplace=True)
    df = pd.merge(df_mean, df_std[['PBM_std','GOM_std','GEM_std','time']], on='time')
    # update time index to be the middle of the month
    df['Year'] = df['time'].dt.year
    df['Month'] = df['time'].dt.month
    # get days in month
    df['Day'] = pd.PeriodIndex(df['time'], freq='M').days_in_month
    df['Day'] = (0.5*df['Day'].astype(float))
    df.drop(columns=['time'], inplace=True)
    df['time'] = pd.to_datetime(df[['Year','Month','Day']])
    df['SiteID'] = SiteID
    return df

def get_annual_means(df, threshold_number=9):
    ''' This function selects years where we have equal to or more than (threshold number) months of data.
        It then calculates annual mean for each year meeting that criteria.'''
    SiteID = df['SiteID'].unique()[0]
    # subset years where we have more than 9 months of data
    counts = df.groupby(by='Year', as_index=False).count()
    yr_list = counts[counts['GEM'] >= threshold_number]['Year'].values
    df = df[df['Year'].isin(yr_list)]
    # calculate annual mean for each year
    df = df.groupby(by='Year', as_index=False).mean(numeric_only=True)
    df['SiteID'] = SiteID

    return df

def get_mean_from_monthly_concentration(df):
    ''' This function groups by month and returns the (annual) mean of the monthly means'''
    SiteID = df['SiteID'].unique()[0]
    df = df.groupby(by='Month', as_index=False).mean(numeric_only=True)
    df = df[['GEM','PBM','GOM']].mean(numeric_only=True)
    df['SiteID'] = SiteID
    # ensure df is a 1 row dataframe
    df = df.to_frame().T
    return df

# ----------------------------------------
# get monthly concentrations for each site
AK03 = get_monthly_concentration(AMNet[AMNet['SiteID'] == 'AK03'])
AK95 = get_monthly_concentration(AMNet[AMNet['SiteID'] == 'AK95'])

# -- note this uses function `get_mean_from_monthly_concentration` defined above
concentration_mean = get_mean_from_monthly_concentration(AK03)
concentration_mean = pd.concat((concentration_mean, get_mean_from_monthly_concentration(AK95)))
concentration_mean = pd.merge(concentration_mean, AMNet_metadata, on='SiteID')

# -- note this uses function `get_annual_means` defined above
concentration_mean_annual = get_annual_means(AK03)
concentration_mean_annual = pd.concat((concentration_mean_annual, get_annual_means(AK95)))
concentration_mean_annual = pd.merge(concentration_mean_annual, AMNet_metadata, on='SiteID')


In [ ]:
# visually inspect monthly values
plt.plot(AK03['time'], AK03['GEM'])
plt.fill_between(AK03['time'], AK03['GEM']-AK03['GEM_std'], AK03['GEM']+AK03['GEM_std'], alpha=0.5)
plt.plot(AK95['time'], AK95['GEM'])
plt.fill_between(AK95['time'], AK95['GEM']-AK95['GEM_std'], AK95['GEM']+AK95['GEM_std'], alpha=0.5)

In [7]:
annual_means_MDN = MDN.groupby(by='siteid', as_index=False).mean(numeric_only=True)
annual_means_MDN = pd.merge(annual_means_MDN, MDN_metadata, on='siteid')
MDN = pd.merge(MDN, MDN_metadata, on='siteid')

In [ ]:
def convert_ppqv_to_ngm3(ds):
    ''' convert ppqv to ng m-3 '''
    return ds*200.59*44.642*1e9

lon_min, lon_max = -169, -139
lat_min, lat_max = 52, 72

myProj = ccrs.NearsidePerspective(central_longitude=-154, central_latitude=62.)
n = 200.
myProj._threshold = myProj._threshold/n  # Set for higher precision of the projection

# standard kwargs for plotting
standard_kwargs={'orientation':'horizontal', 'shrink':0.8, 'pad':0.05,}

# ----------------------------------------
# create the figure
# ----------------------------------------
fig, axs = plt.subplots(nrows=1,ncols=3, subplot_kw={'projection': myProj}, figsize=(7,5))

# ------------------------------
# create subplot for SpeciesConc_Hg0
# ------------------------------
vmin=1
vmax=2+1e-6
step = 0.25
cbar_ticklabels = list(np.arange(vmin, vmax, step))
cbar_ticklabels = [np.round(i,2) for i in cbar_ticklabels]
# check if the label is an integer with modulo division
for i in range(len(cbar_ticklabels)):
    if cbar_ticklabels[i] % 0.5 == 0:
        cbar_ticklabels[i] = np.round(cbar_ticklabels[i],1)
    else:
        cbar_ticklabels[i] = ''

standard_kwargs['ticks'] = np.arange(vmin, vmax, step)

# --- Load data
data = ds['SpeciesConc_Hg0'].sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max)).isel(lev=slice(-3,-1)).mean(dim=['time','lev'])

#data = convert_ppqv_to_ngm3(data)
common_levels = np.arange(1.0, 2.01, 0.25)
ax, im = plot_data2(axs[0], data, title='', vmin=vmin, vmax=vmax)
cbar=fig.colorbar(im, ax=axs[0], **standard_kwargs)
cbar.set_ticklabels(cbar_ticklabels)
cbar.set_label(label=r'GEM [ng m$^{-3}$]', size=10)
# add observations
ax.scatter(concentration_mean['lon'].values, concentration_mean['lat'].values,
           c=concentration_mean['GEM'].values, marker='o', 
           edgecolor='k', linewidth=0.5, s=50,
           vmin=vmin, vmax=vmax, cmap=sargassum, transform=ccrs.PlateCarree(), zorder=6)

ax.set_title(r'Hg$\mathrm{^0}$ Concentration', fontsize=12)


# ------------------------------
# create subplot for WetDep
# ------------------------------

vmin = 0
vmax = 10+1e-6
step = 2 #1
cbar_ticklabels = list(np.arange(vmin, vmax, step))
# check if the label is an integer with modulo division
for i in range(len(cbar_ticklabels)):
    if cbar_ticklabels[i] % 2 == 0:
        cbar_ticklabels[i] = int(cbar_ticklabels[i])
    else:
        cbar_ticklabels[i] = ''

data = ds['Total_Hg_Wet_Dep'].sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max)).mean(dim='time')

ax, im = plot_data2(axs[1], data, title='', vmin=vmin, vmax=vmax)
cbar = fig.colorbar(im, ax=axs[1], **standard_kwargs)
cbar.set_ticks(np.arange(vmin, vmax, step))
cbar.set_ticklabels(cbar_ticklabels)
cbar.set_label(label=r'Wet Deposition [μg m$^{-2}$ y$^{-1}$]', size=10)

obs = annual_means_MDN.copy()
obs = obs[(obs['lat']<=lat_max) & (obs['lat']>=lat_min) & (obs['lon']<=lon_max) & (obs['lon']>=lon_min)]
# add observations
ax.scatter(obs['lon'].values, obs['lat'].values,
           c=obs['HgDep'].values, marker='o', 
           edgecolor='k', linewidth=0.5, s=50,
           vmin=vmin, vmax=vmax, cmap=sargassum, transform=ccrs.PlateCarree(), zorder=6)
ax.set_title('Wet Deposition', fontsize=12)


# ------------------------------
# create subplot for DryDep
# ------------------------------

vmin = 0
vmax = 30+1e-6
step = 5 #2.5
cbar_ticklabels = list(np.arange(vmin, vmax, step))
# check if the label is an integer with modulo division
for i in range(len(cbar_ticklabels)):
    if cbar_ticklabels[i] % 5 == 0:
        cbar_ticklabels[i] = int(cbar_ticklabels[i])
    else:
        cbar_ticklabels[i] = ''

data = ds['Total_Hg_Dry_Dep'].sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max)).mean(dim='time')

ax, im = plot_data2(axs[2], data, title='', vmin=vmin, vmax=vmax)
# add observations
ax.scatter(Obrist['lon'].values, Obrist['lat'].values, c=Obrist['HgDep'].values, marker='o', 
           edgecolor='k', linewidth=0.5, s=50, vmin=vmin, vmax=vmax, cmap=sargassum, transform=ccrs.PlateCarree(), zorder=6)
cbar = fig.colorbar(im, ax=axs[2], **standard_kwargs)
cbar.set_ticks(np.arange(vmin, vmax, step))
cbar.set_ticklabels(cbar_ticklabels)
cbar.set_label(label=r'Dry Deposition [μg m$^{-2}$ y$^{-1}$]', size=10)

ax.set_title('Dry Deposition', fontsize=12)
# ------------------------------
#fig.suptitle('Model Evaluation', fontsize=20, y=0.65)

#plt.tight_layout()
plt.savefig(f'../evaluation/evaluation_maps.png', dpi=1200, bbox_inches='tight')
plt.savefig(f'../evaluation/evaluation_maps.svg', format='svg', bbox_inches='tight')

In [9]:
# now make a table of the observations
# -------------------------------------
evaluation_table = pd.DataFrame({'Category': [],
                    'SiteID': [],
                    'Site Name': [],
                    'Latitude': [],
                    'Longitude': [],
                    'Start Date': [],
                    'End Date': [],
                    'Observed': [],
                    'Lower': [],
                    'Upper': [],
                    'n':[],
                    'Model': [],
                    'Units':[]})

# ------------------------------
mod = ds['SpeciesConc_Hg0'].sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max)).isel(lev=slice(-3,-1)).mean(dim=['time','lev'])
#mod = convert_ppqv_to_ngm3(mod)

for site in concentration_mean['SiteID'].unique():
    # get data
    obs = concentration_mean[concentration_mean['SiteID'] == site]
    obs_annual = concentration_mean_annual[concentration_mean_annual['SiteID'] == site]

    row = pd.DataFrame({'Category': 'GEM',
            'SiteID': [site],
            'Site Name': obs['name'].values,
            'Latitude':  obs['lat'].values,
            'Longitude': obs['lon'].values,
            'Start Date': obs['start_date'].values,
            'End Date': obs['end_date'].values,
            'Observed': obs['GEM'].values,
            'Lower': obs_annual['GEM'].min(),
            'Upper': obs_annual['GEM'].max(),
            'n': int(obs_annual['GEM'].count()),
            'Model': mod.sel(lat=obs['lat'].values, lon=obs['lon'].values, method='nearest').values.item(),
            'Units': ['ng/m3']})
    # add to table
    evaluation_table = pd.concat((evaluation_table, row))

# ------------------------------
mod = ds['Total_Hg_Wet_Dep'].sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max)).mean(dim='time')

for site in MDN['siteid'].unique():
    # get data
    obs = MDN[MDN['siteid'] == site].copy()

    row = pd.DataFrame({'Category': ['Wet Deposition'],
            'SiteID': [site],
            'Site Name': obs['name'].values[0],
            'Latitude':  obs['lat'].values[0],
            'Longitude': obs['lon'].values[0],
            'Start Date': obs['start_date'].values[0],
            'End Date': obs['end_date'].values[0],
            'Observed': np.mean(obs['HgDep']),
            'Lower': np.min(obs['HgDep']), #obs['GEM'].values[0]-data['GEM_std'].values[0],
            'Upper': np.max(obs['HgDep']), #obs['GEM'].values[0]+data['GEM_std'].values[0],
            'n': int(obs['HgDep'].count()),
            'Model': mod.sel(lat=obs['lat'].values[0], lon=obs['lon'].values[0], method='nearest').values.item(),
            'Units': ['μg/m2/yr']})
           
    # add to table
    evaluation_table = pd.concat((evaluation_table, row))

# ------------------------------
mod = ds['Total_Hg_Dry_Dep'].sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max)).mean(dim='time')

for site in Obrist['site'].unique():
    # get data
    obs = Obrist[Obrist['site'] == site].copy()

    row = pd.DataFrame({'Category': ['Dry Deposition'],
            'SiteID': [site],
            'Site Name': obs['Source'].values[0],
            'Latitude':  obs['lat'].values[0],
            'Longitude': obs['lon'].values[0],
            'Start Date': '2014-09',
            'End Date': '2016-09',
            'Observed': np.mean(obs['HgDep']),
            'Lower': np.min(obs['HgDep']), #obs['GEM'].values[0]-data['GEM_std'].values[0],
            'Upper': np.max(obs['HgDep']), #obs['GEM'].values[0]+data['GEM_std'].values[0],
            'n': int(obs['HgDep'].count()),
            'Model': mod.sel(lat=obs['lat'].values[0], lon=obs['lon'].values[0], method='nearest').values.item(),
            'Units': ['μg/m2/yr']})

    # add to table
    evaluation_table = pd.concat((evaluation_table, row))
    
numeric_cols = ['Latitude','Longitude', 'Observed', 'Lower', 'Upper', 'Model', 'n']
evaluation_table[numeric_cols] = evaluation_table[numeric_cols].apply(pd.to_numeric, errors='coerce')
# ------------------------------
dp = 3
round_cols = ['Observed', 'Lower', 'Upper', 'Model']
evaluation_table[round_cols] = evaluation_table[round_cols].round(dp)

# replace Upper and Lower with nan when n == 1
evaluation_table.loc[evaluation_table['n'] == 1, ['Lower', 'Upper']] = np.nan


# remove locations where lat, lon is not within the domain
evaluation_table = evaluation_table[(evaluation_table['Latitude']<=lat_max) & (evaluation_table['Latitude']>=lat_min) & (evaluation_table['Longitude']<=lon_max) & (evaluation_table['Longitude']>=lon_min)]

evaluation_table.to_csv('../evaluation/evaluation_table.csv', index=False)

In [ ]:
evaluation_table

In [ ]:
# calculate normalized mean bias and RMSE for each category
# ------------------------------
for category in evaluation_table['Category'].unique():
    df = evaluation_table[evaluation_table['Category'] == category]
    nmb = (df['Model'] - df['Observed']).sum() / df['Observed'].sum()
    rmse = np.sqrt(((df['Model'] - df['Observed'])**2).sum() / df['Observed'].count())
    print(f'{category}: NMB = {nmb:.2f}, RMSE = {rmse:.2f}')

In [ ]:
# subset sites within domain -- note this excludes sites in coastal SE Alaska
evaluation_table = evaluation_table[(evaluation_table['Latitude'] <= lat_max) & (evaluation_table['Latitude'] >= lat_min) & (evaluation_table['Longitude'] <= lon_max) & (evaluation_table['Longitude'] >= lon_min)]

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

edgecolor = 'k'
model_facecolor='0.3'
obs_facecolor='0.9'
lw=0.5

bar_width = 0.4
space = 0.45

fig = plt.figure(figsize=(1.8,1.5))
ax1 = fig.add_subplot(111)
sel = evaluation_table[evaluation_table['Category'] == 'GEM']
obs = sel['Observed'].values
obs_lower = sel['Lower'].values
obs_upper = sel['Upper'].values
mod = sel['Model'].values
x1 = np.arange(len(obs))
x2 = x1 + space
ax1.bar(x1, obs, yerr=[(obs-obs_lower), (obs_upper-obs)], width=bar_width, label='Observed', color=obs_facecolor, edgecolor=edgecolor, lw=lw,  error_kw=dict(lw=0.5, capsize=2, capthick=0.5))
ax1.bar(x2, mod, width=bar_width, label='Model', color=model_facecolor, edgecolor=edgecolor, lw=lw)
ax1.set_xticks(x1+bar_width/2)
ax1.set_xticklabels(sel['SiteID'].values)
ax1.set_ylim(0.0, 1.8)
ax1.set_yticks(np.arange(0.0, 1.8, 0.4))
ax1.set_ylabel('ng m$^{-3}$')
ax1.set_xlim(np.min(x1)-(0.5*bar_width)-0.1, np.max(x2)+(0.5*bar_width)+0.1)
ax1.set_title('GEM')
# Hide the right and top spines
ax1.spines[['right', 'top', 'bottom']].set_visible(False)
#plt.savefig(f'../evaluation/GEM_bar_chart.png', dpi=800, bbox_inches='tight')
#plt.savefig(f'../evaluation/GEM_bar_chart.svg', format='svg', bbox_inches='tight')

# -- 
fig = plt.figure(figsize=(3.5,1.5))
ax2 = fig.add_subplot(111)
sel = evaluation_table[evaluation_table['Category'] == 'Wet Deposition']
obs = sel['Observed'].values
obs_lower = sel['Lower'].values
obs_upper = sel['Upper'].values
mod = sel['Model'].values
x1 = np.arange(len(obs))
x2 = x1 + space
ax2.bar(x1, obs, yerr=[(obs-obs_lower), (obs_upper-obs)], width=bar_width, label='Observed', color=obs_facecolor, edgecolor=edgecolor, lw=lw, error_kw=dict(lw=0.5, capsize=2, capthick=0.5))
ax2.bar(x2, mod, width=bar_width, label='Model', color=model_facecolor, edgecolor=edgecolor, lw=lw, )
ax2.set_xticks(x1+bar_width/2)
ax2.set_xticklabels(sel['SiteID'].values);
ax2.set_ylim(0, 10)
ax2.set_yticks(np.arange(0, 11, 2))
ax2.set_ylabel("\u03BCg m$^{-2}$ y$^{-1}$ ")
ax2.set_xlim(np.min(x1)-(0.5*bar_width)-0.1, np.max(x2)+(0.5*bar_width)+0.1)
ax2.set_title('Wet Deposition')
# Hide the right and top spines
ax2.spines[['right', 'top', 'bottom']].set_visible(False)
#plt.savefig(f'../evaluation/wet_deposition_bar_chart.png', dpi=800, bbox_inches='tight')
#plt.savefig(f'../evaluation/wet_deposition_bar_chart.svg', format='svg', bbox_inches='tight')

# --
fig = plt.figure(figsize=(1.2,1.5))
ax3 = fig.add_subplot(111)
sel = evaluation_table[evaluation_table['Category'] == 'Dry Deposition']
obs = sel['Observed'].values
obs_lower = sel['Lower'].values
obs_upper = sel['Upper'].values
mod = sel['Model'].values
x1 = np.arange(len(obs))
x2 = x1 + space
ax3.bar(x1, obs, width=bar_width, label='Observed', color=obs_facecolor, edgecolor=edgecolor, lw=lw, error_kw=dict(lw=0.5, capsize=2, capthick=0.5))
ax3.bar(x2, mod, width=bar_width, label='Model', color=model_facecolor, edgecolor=edgecolor, lw=lw,)
ax3.set_xticks(x1+bar_width/2)
ax3.set_xticklabels(sel['SiteID'].values);
ax3.set_ylim(0, 10)
ax3.set_yticks(np.arange(0, 11, 2))
ax3.set_ylabel("\u03BCg m$^{-2}$ y$^{-1}$ ")

ax3.set_xlim(np.min(x1)-(0.5*bar_width)-0.1, np.max(x2)+(0.5*bar_width)+0.1)
ax3.set_title('Dry Deposition')

# Hide the right and top spines
ax3.spines[['right', 'top', 'bottom']].set_visible(False)
#plt.savefig(f'../evaluation/dry_deposition_bar_chart.png', dpi=800, bbox_inches='tight')
#plt.savefig(f'../evaluation/dry_deposition_bar_chart.svg', format='svg', bbox_inches='tight')

In [ ]:
# subset sites within domain -- note this excludes sites in coastal SE Alaska
evaluation_table = evaluation_table[(evaluation_table['Latitude'] <= lat_max) & (evaluation_table['Latitude'] >= lat_min) & (evaluation_table['Longitude'] <= lon_max) & (evaluation_table['Longitude'] >= lon_min)]

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# turn grid off
plt.rcParams['axes.grid'] = False

# set xtick length to 0
plt.rcParams['xtick.major.size'] = 0
plt.rcParams['xtick.minor.size'] = 0

edgecolor = 'k'
model_facecolor='0.3'
obs_facecolor='0.9'
lw=0.5
title_pad = 8
label_pad = -1
y_pad_rel = 0.01

bar_width = 0.4
space = 0.45

fig = plt.figure(figsize=(7,1.75))
gs = GridSpec(1, 3, figure=fig, width_ratios=[0.25,0.625,0.125])

ax1 = fig.add_subplot(gs[0])
sel = evaluation_table[evaluation_table['Category'] == 'GEM']
obs = sel['Observed'].values
obs_lower = sel['Lower'].values
obs_upper = sel['Upper'].values
mod = sel['Model'].values
x1 = np.arange(len(obs))
x2 = x1 + space
ax1.bar(x1, obs, yerr=[(obs-obs_lower), (obs_upper-obs)], width=bar_width, label='Observed', color=obs_facecolor, edgecolor=edgecolor, lw=lw,  error_kw=dict(lw=0.5, capsize=2, capthick=0.5))
ax1.bar(x2, mod, width=bar_width, label='Model', color=model_facecolor, edgecolor=edgecolor, lw=lw)
ax1.set_xticks(x1+bar_width/2)
ax1.set_xticklabels(sel['SiteID'].values)
ax1.set_ylim(-0.018, 1.8)
ax1.set_yticks(np.arange(0.0, 1.8, 0.4))
ax1.set_ylabel('ng m$^{-3}$', labelpad=label_pad+3)
ax1.set_xlim(np.min(x1)-(0.5*bar_width)-0.1, np.max(x2)+(0.5*bar_width)+0.1)
ax1.set_title('Hg$^\mathrm{0}$ Concentration', pad=title_pad)
# Hide the right and top spines
ax1.spines[['right', 'top', 'bottom']].set_visible(False)

# -- 
ax2 = fig.add_subplot(gs[1])
sel = evaluation_table[evaluation_table['Category'] == 'Wet Deposition']
obs = sel['Observed'].values
obs_lower = sel['Lower'].values
obs_upper = sel['Upper'].values
mod = sel['Model'].values
x1 = np.arange(len(obs))
x2 = x1 + space
ax2.bar(x1, obs, yerr=[(obs-obs_lower), (obs_upper-obs)], width=bar_width, label='Observed', color=obs_facecolor, edgecolor=edgecolor, lw=lw, error_kw=dict(lw=0.5, capsize=2, capthick=0.5))
ax2.bar(x2, mod, width=bar_width, label='Model', color=model_facecolor, edgecolor=edgecolor, lw=lw, )
ax2.set_xticks(x1+bar_width/2)
ax2.set_xticklabels(sel['SiteID'].values);
ax2.set_ylim(-0.1, 10)
ax2.set_yticks(np.arange(0, 11, 2))
ax2.set_ylabel("\u03BCg m$^{-2}$ y$^{-1}$", labelpad=label_pad)
ax2.set_xlim(np.min(x1)-(0.5*bar_width)-0.1, np.max(x2)+(0.5*bar_width)+0.1)
ax2.set_title('Wet Deposition', pad=title_pad)
# Hide the right and top spines
ax2.spines[['right', 'top', 'bottom']].set_visible(False)

# --
ax3 = fig.add_subplot(gs[2])
sel = evaluation_table[evaluation_table['Category'] == 'Dry Deposition']
obs = sel['Observed'].values
obs_lower = sel['Lower'].values
obs_upper = sel['Upper'].values
mod = sel['Model'].values
x1 = np.arange(len(obs))
x2 = x1 + space
ax3.bar(x1, obs, width=bar_width, label='Observed', color=obs_facecolor, edgecolor=edgecolor, lw=lw, error_kw=dict(lw=0.5, capsize=2, capthick=0.5))
ax3.bar(x2, mod, width=bar_width, label='Model', color=model_facecolor, edgecolor=edgecolor, lw=lw,)
ax3.set_xticks(x1+bar_width/2)
ax3.set_xticklabels(sel['SiteID'].values);

ax3.set_ylim(-0.1, 10)
ax3.set_yticks(np.arange(0, 11, 2))
ax3.set_ylabel("\u03BCg m$^{-2}$ y$^{-1}$", labelpad=label_pad)

ax3.set_xlim(np.min(x1)-(0.5*bar_width)-0.1, np.max(x2)+(0.5*bar_width)+0.1)
ax3.set_title('Dry Deposition', pad=title_pad)

# Hide the right and top spines
ax3.spines[['right', 'top', 'bottom']].set_visible(False)
plt.savefig(f'../evaluation/evaluation_bar_chart.png', dpi=1200, bbox_inches='tight')
plt.savefig(f'../evaluation/evaluation_bar_chart.svg', format='svg')

In [16]:
masks = xr.open_dataset('../misc/masks.nc')
masks = masks.sel(lat=slice(ds['lat'].min(), ds['lat'].max()), lon=slice(ds['lon'].min(), ds['lon'].max()))
ds = xr.merge([ds, masks])
ds['Met_NOTOCEAN'] = 1-ds['Met_FROCEAN']

In [ ]:
# 1. for each domain, print deposition in kg and ug/m2/yr
# ----------------------------------------
dep_var = 'Total_Hg_Wet_Dep'
domain = 'Alaska'
mask_var = None

def get_annual_deposition(ds, dep_var='Total_Hg_Wet_Dep', domain='Alaska', mask_var=None, output_units = 'kg'):
    
    if mask_var:
        mask = ds[mask_var]
        ds = ds.where(mask)

    # subset to domain
    if (domain != '') or (domain != None):
        #print(f'subsetting domain to {domain}')
        ds = ds.sel(lat=slice(domain_bounds[domain]['lat_min'], domain_bounds[domain]['lat_max']), lon=slice(domain_bounds[domain]['lon_min'], domain_bounds[domain]['lon_max']))

    assert ds[dep_var].units == 'μg m-2 yr-1', f'units on {dep_var} are not μg m-2 yr-1'
    ds_out = (ds[dep_var]*(ds.time.dt.days_in_month/365.25)).sum('time')
    
    if output_units == 'kg':
        # convert from μg m-2 yr-1 to kg
        ds_out = (ds_out*ds['AREA']*1e-9).sum().values
    elif output_units in ['μg/m2/yr', 'ug/m2/yr']:
        ds_out = (ds_out*ds['AREA']).sum().values/ds['AREA'].sum().values
    else:
        raise ValueError('output_units must be kg or μg/m2/yr')
    return ds_out

table = pd.DataFrame()
for domain in ['Alaska', 'YKD', 'IKU']:
    for mask_name in ['', 'Met_NOTOCEAN']:
        row = {'domain': domain}
        if mask_name == '':
            #print('no mask')
            row['mask'] = 'none'
        else:
            #print(f'masked by {mask_name}')
            row['mask'] = 'land only'
        for v, col_name in zip(['Total_Hg_Deposition', 'Total_Hg_Wet_Dep', 'Total_Hg_Dry_Dep', 'FluxHg0fromAirToOcean'], ['total dep', 'wet dep', 'dry dep', 'ocean uptake']):
            row[f'{col_name} [kg]'] = get_annual_deposition(ds, dep_var=v, domain=domain, mask_var=mask_name, output_units='kg')
            row[f'{col_name} [μg/m2/yr]'] = get_annual_deposition(ds, dep_var=v, domain=domain, mask_var=mask_name, output_units='μg/m2/yr')

        table = pd.concat((table, pd.DataFrame(row, index=[0])))

# display table with 1 decimal place
table = table.round(1)

# sort table by mask and domain
# set categorical order for domain based on area
table['domain'] = pd.Categorical(table['domain'], ['Alaska', 'YKD', 'IKU'])
table['mask'] = pd.Categorical(table['mask'], ['none', 'land only'])
table = table.sort_values(by=['mask','domain'])
table

In [ ]:
table